In [1]:
import pandas as pd
import numpy as np
#data = pd.read_csv('drive/MyDrive/ColabNotebooks/MPR1/Files/ner_dataset.csv', encoding= 'unicode_escape')
data = pd.read_csv('drive/MyDrive/ColabNotebooks/MPR1/Files/kaggle_at_home.csv', encoding= 'unicode_escape')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')] # remove Unnamed column


In [2]:
data = data.head(10000)
data['Word'].fillna(" ", inplace=True)
data.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,Sentence #,Word,POS,Tag
0,Sentence: 1,,NNS,O
1,NaN,FOURTH,NNS,O
2,NaN,SECTION,NNS,O
3,NaN,,NNS,O
4,NaN,CASE,NNS,O


In [3]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok


token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [4]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data.head()

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,,NNS,O,117,2
1,NaN,FOURTH,NNS,O,165,2
2,NaN,SECTION,NNS,O,200,2
3,NaN,,NNS,O,117,2
4,NaN,CASE,NNS,O,209,2


In [5]:
# Fill na
data_fillna = data.fillna(method='ffill', axis=0)
# Groupby and collect columns
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))
# Visualise data
data_group.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[ , FOURTH, SECTION, , CASE, OF, POHOSKA, v.,...","[NNS, NNS, NNS, NNS, NNS, NNS, NNS, NNS, NNS, ...","[O , O , O , O , O , O , O , O , O , O , O , O...","[117, 165, 200, 117, 209, 217, 152, 107, 117, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, ..."
1,Sentence: 10,"[ , 4, .]","[NNS, NNS, NNS]","[O , B CARDINAL, O ]","[117, 144, 47]","[2, 0, 2]"
2,Sentence: 11,"[ , On, 30, August, 2010, the, President, of, ...","[NNS, NNS, NNS, NNS, NNS, NNS, NNS, NNS, NNS, ...","[O , O , B DATE, I DATE, I DATE, O , O , O , B...","[117, 98, 105, 122, 35, 123, 199, 137, 123, 14...","[2, 2, 6, 12, 12, 2, 2, 2, 9, 15, 15, 2, 2, 2,..."
3,Sentence: 12,"[ , THE, FACTS, , I., , THE, CIRCUMSTANCES, ...","[NNS, NNS, NNS, NNS, NNS, NNS, NNS, NNS, NNS, ...","[O , O , B ORG, O , O , O , O , O , O , O , O ...","[117, 147, 126, 117, 69, 117, 147, 203, 217, 1...","[2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2]"
4,Sentence: 13,"[ , The, applicant, was, born, in, 1951, and, ...","[NNS, NNS, NNS, NNS, NNS, NNS, NNS, NNS, NNS, ...","[O , O , O , O , O , O , B DATE, O , O , O , O ]","[117, 23, 74, 145, 99, 92, 79, 59, 214, 92, 47]","[2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2]"


In [6]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [7]:
def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    #Pad tokens (X var)    
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= n_tag - 1)
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )
    
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 13 
train_tokens length: 13 
test_tokens length: 2 
test_tags: 2 
val_tokens: 5 
val_tags: 5


In [8]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model

In [9]:
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [10]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)
print('input_dim: ', input_dim, '\noutput_dim: ', output_dim, '\ninput_length: ', input_length, '\nn_tags: ', n_tags)

input_dim:  228 
output_dim:  64 
input_length:  99 
n_tags:  16


In [11]:
def get_bilstm_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Add LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))

    #Optimiser 
    # adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [12]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [13]:
results = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 99, 64)            14592     
                                                                 
 bidirectional (Bidirectiona  (None, 99, 128)          66048     
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 99, 64)            49408     
                                                                 
 time_distributed (TimeDistr  (None, 99, 16)           1040      
 ibuted)                                                         
                                                                 
Total params: 131,088
Trainable params: 131,088
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 436ms/st